In [1]:
# =============================================================
# 🔁 OPEN MIRRORING CONFIGURATION (Auto-Healing Simulation Mode)
# =============================================================

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import functions as F

class OpenMirroringManager:
    """
    Handles configuration of Open Mirroring in Microsoft Fabric Lakehouse.
    Falls back to simulation mode outside Fabric, creating demo data if needed.
    """

    def __init__(self, spark: SparkSession, database_name="ParkingDataLakehouse"):
        self.spark = spark
        self.database_name = database_name

    # ---------------------------------------------------------
    # Step 1 – Enable Fabric Mirroring (if supported)
    # ---------------------------------------------------------
    def configure_open_mirroring(self):
        print("🚀 STARTING OPEN MIRRORING SETUP")
        print("🔄 CONFIGURING OPEN MIRRORING FOR DELTA REPLICATION...")

        try:
            cluster_info = self.spark.conf.get(
                "spark.databricks.clusterUsageTags.clusterType", ""
            ).lower()

            if "fabric" in cluster_info or "synapse" in cluster_info:
                print("🧠 Detected Microsoft Fabric runtime.")
                self.spark.sql(f"""
                    ALTER DATABASE {self.database_name}
                    SET DBPROPERTIES ('enable_mirroring'='true')
                """)
                print(f"✅ Mirroring enabled for {self.database_name}")
            else:
                print("🧩 Non-Fabric runtime detected — switching to simulation mode.")
                print("ℹ️ Skipping ALTER DATABASE (Fabric-only feature).")
        except Exception as e:
            print(f"⚠️ Mirroring note: {e}")
            print("⚙️ Continuing in simulation mode...")

    # ---------------------------------------------------------
    # Step 2 – Ensure mock tables exist (create if missing)
    # ---------------------------------------------------------
    def _ensure_table_exists(self, table_name: str):
        full_name = f"{self.database_name}.{table_name}"
        try:
            self.spark.table(full_name)
            print(f"✅ Found source table: {full_name}")
            return True
        except Exception:
            print(f"ℹ️ Table not found: {full_name} — creating mock data...")

            # Create mock DataFrame per table
            if table_name == "ParkingSensorData":
                df = self.spark.createDataFrame([
                    ("Zone A", True, "2025-10-11 08:00", True, "tx001"),
                    ("Zone B", False, "2025-10-11 08:05", True, "tx002"),
                ], ["parking_zone", "occupancy_status", "timestamp",
                    "blockchain_verified", "blockchain_tx_id"])

            elif table_name == "TrafficCameraData":
                df = self.spark.createDataFrame([
                    ("Cam-1", "HIGH", 0.85, 34, "2025-10-11 08:00"),
                    ("Cam-2", "MEDIUM", 0.50, 20, "2025-10-11 08:05"),
                ], ["camera_id", "congestion_level", "traffic_density",
                    "vehicle_count", "timestamp"])

            elif table_name == "YOLOProcessedData":
                df = self.spark.createDataFrame([
                    ("Cam-1", 0.92, "SUCCESS", "2025-10-11 08:00"),
                    ("Cam-2", 0.78, "SUCCESS", "2025-10-11 08:05"),
                ], ["camera_id", "processing_confidence",
                    "processing_status", "processed_at"])
            else:
                schema = StructType([StructField("id", StringType())])
                df = self.spark.createDataFrame([], schema)

            # Ensure database exists
            self.spark.sql(f"CREATE DATABASE IF NOT EXISTS {self.database_name}")
            df.write.mode("overwrite").format("delta").saveAsTable(full_name)
            print(f"✅ Mock table created: {full_name}")
            return True

    # ---------------------------------------------------------
    # Step 3 – Simulate Mirroring by copying to local schema
    # ---------------------------------------------------------
    def simulate_replication(self, table_names=None):
        if table_names is None:
            table_names = ["ParkingSensorData",
                           "TrafficCameraData",
                           "YOLOProcessedData"]

        print("\n🧱 SIMULATING OPEN MIRRORING (Local Copy Mode)...")
        self.spark.sql("CREATE DATABASE IF NOT EXISTS MirroredData")

        for table in table_names:
            src_table = f"{self.database_name}.{table}"
            dst_table = f"MirroredData.{table}"

            self._ensure_table_exists(table)

            try:
                df = self.spark.table(src_table)
                df.write.mode("overwrite").format("delta").saveAsTable(dst_table)
                print(f"✅ Mirrored table created: {dst_table} ({df.count()} rows)")
            except Exception as e:
                print(f"⚠️ Skipping {table}: {e}")

        print("🏁 Mirroring simulation completed successfully.\n")

# ---------------------------------------------------------
# ✅ Optional standalone execution
# ---------------------------------------------------------
if __name__ == "__main__":
    spark = SparkSession.builder.appName("OpenMirroringSetup").getOrCreate()
    mgr = OpenMirroringManager(spark)
    mgr.configure_open_mirroring()
    mgr.simulate_replication()
    print("✅ Open Mirroring setup completed successfully.")


StatementMeta(, 36585711-9b79-4c83-8c72-643e2aa1302f, 3, Finished, Available, Finished)

🚀 STARTING OPEN MIRRORING SETUP
🔄 CONFIGURING OPEN MIRRORING FOR DELTA REPLICATION...
🧩 Non-Fabric runtime detected — switching to simulation mode.
ℹ️ Skipping ALTER DATABASE (Fabric-only feature).

🧱 SIMULATING OPEN MIRRORING (Local Copy Mode)...
✅ Found source table: ParkingDataLakehouse.ParkingSensorData
✅ Mirrored table created: MirroredData.ParkingSensorData (2 rows)
✅ Found source table: ParkingDataLakehouse.TrafficCameraData
✅ Mirrored table created: MirroredData.TrafficCameraData (2 rows)
✅ Found source table: ParkingDataLakehouse.YOLOProcessedData
✅ Mirrored table created: MirroredData.YOLOProcessedData (2 rows)
🏁 Mirroring simulation completed successfully.

✅ Open Mirroring setup completed successfully.


In [1]:
# =============================================================
# 🔁 OPEN MIRRORING CONFIGURATION + SAFE EXPORT (Auto-Healing Mode)
# =============================================================
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import functions as F
import os, shutil, pandas as pd

class OpenMirroringManager:
    """
    Handles configuration of Open Mirroring in Microsoft Fabric Lakehouse.
    Falls back to simulation mode outside Fabric, creating demo data if needed.
    Now includes automatic Safe Exporter for CSV download.
    """

    def __init__(self, spark: SparkSession, database_name="ParkingDataLakehouse", mirror_name="MirroredData"):
        self.spark = spark
        self.database_name = database_name
        self.mirror_name = mirror_name
        self.output_dir = "/tmp/lakehouse_export_local"
        os.makedirs(self.output_dir, exist_ok=True)

    # ---------------------------------------------------------
    # Step 1 – Enable Fabric Mirroring (if supported)
    # ---------------------------------------------------------
    def configure_open_mirroring(self):
        print("🚀 STARTING OPEN MIRRORING SETUP")
        print("🔄 CONFIGURING OPEN MIRRORING FOR DELTA REPLICATION...")

        try:
            cluster_info = self.spark.conf.get(
                "spark.databricks.clusterUsageTags.clusterType", ""
            ).lower()

            if "fabric" in cluster_info or "synapse" in cluster_info:
                print("🧠 Detected Microsoft Fabric runtime.")
                self.spark.sql(f"""
                    ALTER DATABASE {self.database_name}
                    SET DBPROPERTIES ('enable_mirroring'='true')
                """)
                print(f"✅ Mirroring enabled for {self.database_name}")
            else:
                print("🧩 Non-Fabric runtime detected — switching to simulation mode.")
                print("ℹ️ Skipping ALTER DATABASE (Fabric-only feature).")
        except Exception as e:
            print(f"⚠️ Mirroring note: {e}")
            print("⚙️ Continuing in simulation mode...")

    # ---------------------------------------------------------
    # Step 2 – Ensure mock tables exist (create if missing)
    # ---------------------------------------------------------
    def _ensure_table_exists(self, table_name: str):
        full_name = f"{self.database_name}.{table_name}"
        try:
            self.spark.table(full_name)
            print(f"✅ Found source table: {full_name}")
            return True
        except Exception:
            print(f"ℹ️ Table not found: {full_name} — creating mock data...")

            # Create mock DataFrame per table
            if table_name == "ParkingSensorData":
                df = self.spark.createDataFrame([
                    ("Zone A", True, "2025-10-11 08:00", True, "tx001"),
                    ("Zone B", False, "2025-10-11 08:05", True, "tx002"),
                ], ["parking_zone", "occupancy_status", "timestamp",
                    "blockchain_verified", "blockchain_tx_id"])

            elif table_name == "TrafficCameraData":
                df = self.spark.createDataFrame([
                    ("Cam-1", "HIGH", 0.85, 34, "2025-10-11 08:00"),
                    ("Cam-2", "MEDIUM", 0.50, 20, "2025-10-11 08:05"),
                ], ["camera_id", "congestion_level", "traffic_density",
                    "vehicle_count", "timestamp"])

            elif table_name == "YOLOProcessedData":
                df = self.spark.createDataFrame([
                    ("Cam-1", 0.92, "SUCCESS", "2025-10-11 08:00"),
                    ("Cam-2", 0.78, "SUCCESS", "2025-10-11 08:05"),
                ], ["camera_id", "processing_confidence",
                    "processing_status", "processed_at"])
            else:
                schema = StructType([StructField("id", StringType())])
                df = self.spark.createDataFrame([], schema)

            # Ensure database exists
            self.spark.sql(f"CREATE DATABASE IF NOT EXISTS {self.database_name}")
            df.write.mode("overwrite").format("delta").saveAsTable(full_name)
            print(f"✅ Mock table created: {full_name}")
            return True

    # ---------------------------------------------------------
    # Step 3 – Simulate Mirroring by copying to local schema
    # ---------------------------------------------------------
    def simulate_replication(self, table_names=None):
        if table_names is None:
            table_names = ["ParkingSensorData",
                           "TrafficCameraData",
                           "YOLOProcessedData"]

        print("\n🧱 SIMULATING OPEN MIRRORING (Local Copy Mode)...")
        self.spark.sql(f"CREATE DATABASE IF NOT EXISTS {self.mirror_name}")

        for table in table_names:
            src_table = f"{self.database_name}.{table}"
            dst_table = f"{self.mirror_name}.{table}"

            self._ensure_table_exists(table)

            try:
                df = self.spark.table(src_table)
                df.write.mode("overwrite").format("delta").saveAsTable(dst_table)
                print(f"✅ Mirrored table created: {dst_table} ({df.count()} rows)")
            except Exception as e:
                print(f"⚠️ Skipping {table}: {e}")

        print("🏁 Mirroring simulation completed successfully.\n")

    # ---------------------------------------------------------
    # Step 4 – Export all tables (source + mirrored) safely
    # ---------------------------------------------------------
    def export_all_data(self):
        print("📦 Starting Safe Lakehouse Export (dbo-style)...")

        all_dbs = [self.database_name, self.mirror_name]
        for db in all_dbs:
            try:
                tables = [f"{db}.{t.name}" for t in self.spark.catalog.listTables(db)]
                for table in tables:
                    try:
                        print(f"📤 Exporting {table} ...")
                        df = self.spark.table(table)
                        pdf = df.limit(100000).toPandas()
                        csv_path = os.path.join(self.output_dir, f"{table.replace('.', '_')}.csv")
                        pdf.to_csv(csv_path, index=False)
                        print(f"✅ Saved {csv_path} ({len(pdf)} rows)")
                    except Exception as e:
                        print(f"⚠️ Skipping {table}: {e}")
            except Exception as e:
                print(f"⚠️ Could not list tables in {db}: {e}")

        zip_path = f"{self.output_dir}.zip"
        shutil.make_archive(self.output_dir, "zip", self.output_dir)
        print(f"🏁 Export completed successfully → {zip_path}")
        return zip_path


# ---------------------------------------------------------
# ✅ Optional standalone execution
# ---------------------------------------------------------
if __name__ == "__main__":
    spark = SparkSession.builder.appName("OpenMirroringWithExport").getOrCreate()
    mgr = OpenMirroringManager(spark)
    mgr.configure_open_mirroring()
    mgr.simulate_replication()
    zip_path = mgr.export_all_data()
    print(f"📦 All tables exported → {zip_path}")


StatementMeta(, e821c89f-0dfa-43fe-abbd-1632ed37e285, 3, Finished, Available, Finished)

🚀 STARTING OPEN MIRRORING SETUP
🔄 CONFIGURING OPEN MIRRORING FOR DELTA REPLICATION...
🧩 Non-Fabric runtime detected — switching to simulation mode.
ℹ️ Skipping ALTER DATABASE (Fabric-only feature).

🧱 SIMULATING OPEN MIRRORING (Local Copy Mode)...
✅ Found source table: ParkingDataLakehouse.ParkingSensorData
✅ Mirrored table created: MirroredData.ParkingSensorData (2 rows)
✅ Found source table: ParkingDataLakehouse.TrafficCameraData
✅ Mirrored table created: MirroredData.TrafficCameraData (2 rows)
✅ Found source table: ParkingDataLakehouse.YOLOProcessedData
✅ Mirrored table created: MirroredData.YOLOProcessedData (2 rows)
🏁 Mirroring simulation completed successfully.

📦 Starting Safe Lakehouse Export (dbo-style)...
📤 Exporting ParkingDataLakehouse.parkingsensordata ...
✅ Saved /tmp/lakehouse_export_local/ParkingDataLakehouse_parkingsensordata.csv (2 rows)
📤 Exporting ParkingDataLakehouse.trafficcameradata ...
✅ Saved /tmp/lakehouse_export_local/ParkingDataLakehouse_trafficcameradata.cs